In [1]:
import tensorflow as tf
import os
import numpy as np

2024-05-16 22:13:39.295340: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 22:13:40.160499: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def generate_triplets(directory, batch_size=32):
    while True:
        anchor_images, positive_images, negative_images = [], [], []
        classes = [folder for folder in os.listdir(directory) if os.path.isdir(os.path.join(directory, folder))]
        class_paths = {cls: os.path.join(directory, cls) for cls in classes}
        valid_anchor_classes = [cls for cls in classes if len(os.listdir(class_paths[cls])) > 1]
        valid_negative_classes = [cls for cls in classes if len(os.listdir(class_paths[cls])) >= 1]

        # Ensure that the batch is completely filled
        while len(anchor_images) < batch_size:
            if not valid_anchor_classes:
                continue  # Skip if no valid anchor classes available

            anchor_class = np.random.choice(valid_anchor_classes)
            anchor_path = class_paths[anchor_class]

            # Selecting anchor and positive images
            anchor_positive = np.random.choice(os.listdir(anchor_path), 2, replace=False)
            anchor_img_path = os.path.join(anchor_path, anchor_positive[0])
            positive_img_path = os.path.join(anchor_path, anchor_positive[1])

            # Reading and processing images
            anchor_img = tf.io.read_file(anchor_img_path)
            positive_img = tf.io.read_file(positive_img_path)
            anchor_img = tf.image.decode_jpeg(anchor_img, channels=3)
            positive_img = tf.image.decode_jpeg(positive_img, channels=3)
            anchor_img = tf.image.resize(anchor_img, (224, 224))
            positive_img = tf.image.resize(positive_img, (224, 224))

            # Selecting a negative class and image
            if not valid_negative_classes:
                continue  # Skip if no valid negative classes available
            negative_class = np.random.choice([cls for cls in valid_negative_classes if cls != anchor_class])
            negative_path = class_paths[negative_class]
            negative_img_path = os.path.join(negative_path, np.random.choice(os.listdir(negative_path)))
            negative_img = tf.io.read_file(negative_img_path)
            negative_img = tf.image.decode_jpeg(negative_img, channels=3)
            negative_img = tf.image.resize(negative_img, (224, 224))

            # Appending to the batch
            anchor_images.append(anchor_img)
            positive_images.append(positive_img)
            negative_images.append(negative_img)

        yield [np.array(anchor_images), np.array(positive_images), np.array(negative_images)], np.zeros((batch_size, 1))

# Test the generator
for data, _ in generate_triplets('/workspaces/finetune/combined_classes', batch_size=32):
    print(data[0].shape, data[1].shape, data[2].shape)  # Should print shapes (32, 224, 224, 3) for each
    break


(32, 224, 224, 3) (32, 224, 224, 3) (32, 224, 224, 3)


In [3]:
from tensorflow.keras.callbacks import Callback

class CustomPrintCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(f"End of epoch {epoch + 1}")
        print(f"Loss: {logs.get('loss')}")

# Create the callback instance
custom_print_callback = CustomPrintCallback()

In [4]:
# Assuming your model and triplet loss function are correctly defined as previously described
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard

import datetime
import os

%load_ext tensorboard
%tensorboard --logdir logs/fit

# Directory where the logs will be written:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Create a dataset from the generator with an output signature
output_signature = (
    (
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),  # Anchor images
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),  # Positive images
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32)   # Negative images
    ),
    tf.TensorSpec(shape=(None, 1), dtype=tf.float32)                # Dummy labels
)

# Create the dataset
dataset = tf.data.Dataset.from_generator(
    generator=lambda: generate_triplets('/workspaces/finetune/combined_classes', batch_size=32),
    output_signature=output_signature
)

# Define your base CNN and triplet loss
def create_base_cnn(input_shape):
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
    from tensorflow.keras import Input, Model

    input = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(input)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    return Model(inputs=input, outputs=x)

from tensorflow.keras.layers import Lambda
import tensorflow.keras.backend as K


base_model = create_base_cnn((224, 224, 3))
anchor_input = Input((224, 224, 3), name='anchor_input')
positive_input = Input((224, 224, 3), name='positive_input')
negative_input = Input((224, 224, 3), name='negative_input')

anchor_embedding = base_model(anchor_input)
positive_embedding = base_model(positive_input)
negative_embedding = base_model(negative_input)

outputs = [anchor_embedding, positive_embedding, negative_embedding]


def triplet_loss(alpha=0.2):
    def loss(y_true, y_pred):
        anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
        pos_dist = K.sum(K.square(anchor - positive), axis=-1)
        neg_dist = K.sum(K.square(anchor - negative), axis=-1)
        basic_loss = pos_dist - neg_dist + alpha
        return K.maximum(basic_loss, 0.0)
    return loss

def triplet_loss_layer(vectors):
    anchor, positive, negative = vectors
    return triplet_loss(alpha=0.2)(None, [anchor, positive, negative])

# Using the Lambda layer
triplet_layer = Lambda(triplet_loss_layer, output_shape=(1,))

# Assuming anchor_embedding, positive_embedding, and negative_embedding are your model outputs
outputs = triplet_layer([anchor_embedding, positive_embedding, negative_embedding])

# Create the final model
model = Model(inputs=[anchor_input, positive_input, negative_input], outputs=outputs)

# Compile the model with an optimizer and the defined triplet loss
model.compile(optimizer='adam', loss='mean_squared_error')  # Here, actual loss computation is handled within the layer

model.fit(
    dataset,
    steps_per_epoch=100,
    epochs=10,
    callbacks=[custom_print_callback, tensorboard_callback]  # Add TensorBoard here
)


Launching TensorBoard...

Epoch 1/10


2024-05-16 22:13:49.674424: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was ((tf.float32, tf.float32, tf.float32), tf.float32), but the yielded element was ([array([[[[107.98214 , 145.40051 , 157.67348 ],
         [119.13776 , 148.70409 , 160.27806 ],
         [120.34694 , 141.56888 , 162.51787 ],
         ...,
         [157.06375 , 147.06375 , 120.7781  ],
         [162.17337 , 152.17337 , 127.17338 ],
         [154.51529 , 142.65811 , 118.58671 ]],

        [[115.64541 , 146.49745 , 170.03317 ],
         [121.66072 , 146.8546  , 171.70409 ],
         [128.39285 , 147.21175 , 172.52042 ],
         ...,
         [160.31886 , 150.31886 , 123.65052 ],
         [161.93605 , 151.93605 , 124.93605 ],
         [158.6991  , 145.69398 , 119.12511 ]],

        [[193.94899 , 178.34184 , 216.07144 ],
         [168.86736 , 149.01021 , 195.77551 ],
         [138.62245 , 1

InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was ((tf.float32, tf.float32, tf.float32), tf.float32), but the yielded element was ([array([[[[107.98214 , 145.40051 , 157.67348 ],
         [119.13776 , 148.70409 , 160.27806 ],
         [120.34694 , 141.56888 , 162.51787 ],
         ...,
         [157.06375 , 147.06375 , 120.7781  ],
         [162.17337 , 152.17337 , 127.17338 ],
         [154.51529 , 142.65811 , 118.58671 ]],

        [[115.64541 , 146.49745 , 170.03317 ],
         [121.66072 , 146.8546  , 171.70409 ],
         [128.39285 , 147.21175 , 172.52042 ],
         ...,
         [160.31886 , 150.31886 , 123.65052 ],
         [161.93605 , 151.93605 , 124.93605 ],
         [158.6991  , 145.69398 , 119.12511 ]],

        [[193.94899 , 178.34184 , 216.07144 ],
         [168.86736 , 149.0102

2024-05-16 22:13:52.331148: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was ((tf.float32, tf.float32, tf.float32), tf.float32), but the yielded element was ([array([[[[142.90625  , 152.90625  , 141.90625  ],
         [146.8125   , 146.8125   , 136.8125   ],
         [168.25     , 165.25     , 156.25     ],
         ...,
         [ 15.0625   ,  48.0625   ,  53.0625   ],
         [ 15.1875   ,  48.1875   ,  53.1875   ],
         [ 17.25     ,  50.25     ,  57.25     ]],

        [[153.0625   , 163.0625   , 152.0625   ],
         [148.3125   , 155.6875   , 145.       ],
         [127.       , 131.03125  , 119.6875   ],
         ...,
         [ 17.9375   ,  50.9375   ,  55.9375   ],
         [ 18.03125  ,  51.03125  ,  58.03125  ],
         [ 21.125    ,  54.125    ,  61.125    ]],

        [[104.1875   , 116.1875   , 104.1875   ],
         [124.5      , 136.5   